In [1]:
import sys
sys.path.append("/home/ly/workspace/mmsa")
seed = 1938
import numpy as np
import torch
from torch import nn
from torch import optim

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from models.bigru_base_rcnn import *
from utils.train import *
from typing import *
from utils.load_yelp import *
from utils.dataset import *
from utils.train import *
from utils.train import *

config

{'task': 'CLS',
 'embedding_dim': 100,
 'embedding': None,
 'freeze_embedding': True,
 'text_hidden_size': 100,
 'text_layers': 1,
 'bias_init': 1.0,
 'img_input_size': 2048,
 'img_output_size': 100,
 'img_num': 3,
 'output_size': 5,
 'uniform_bound': 0.1,
 'dropout': 0.5}

In [2]:
%%time
batch_size = 64
workers = 4
train_set, valid_set, test_set= load_glove_data(split811data, config)
train_loader, valid_loader, test_loader = get_loader(batch_size, workers, get_collate_fn(config), train_set, valid_set, test_set)
model = Model(config).cuda()
loss = nn.CrossEntropyLoss()
print(get_parameter_number(model))

_interval = 5
lr = 1e-3
epoches = 50
stoping_step = 5
optimizer = get_regal_optimizer(model, optim.AdamW, lr)

viz = get_Visdom()
batch_loss_drawer = VisdomScalar(viz, f"batch_loss interval:{_interval}")
epoch_loss_drawer = VisdomScalar(viz, f"Train and valid loss", 2)
acc_drawer = VisdomScalar(viz, "Train and valid accuracy", 2)
text_writer = VisdomTextWriter(viz, "Training")

batch_loss = []
train_loss = []
valid_loss = []
train_acc = []
valid_acc = []

res, model = train_visdom_v2(model, optimizer, loss, viz, train_loader,
                          valid_loader, epoches, batch_loss, batch_loss_drawer,
                          train_loss, valid_loss, epoch_loss_drawer,
                          train_acc, valid_acc, acc_drawer, text_writer,
                         _interval=_interval, early_stop=stoping_step)
eval_model(model, test_loader, loss)

/home/ly/miniconda3/envs/torch1.7/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Setting up a new session...


{'Total': 4611005, 'Trainable': 327605}


No 1 epoch: 554it [01:27,  6.32it/s]
No 2 epoch: 554it [01:12,  7.62it/s]
No 3 epoch: 554it [01:13,  7.58it/s]
No 4 epoch: 554it [01:13,  7.51it/s]
No 5 epoch: 554it [01:14,  7.43it/s]
No 6 epoch: 554it [01:14,  7.39it/s]
No 7 epoch: 554it [01:14,  7.45it/s]
No 8 epoch: 554it [01:13,  7.49it/s]
No 9 epoch: 554it [01:15,  7.38it/s]
No 10 epoch: 554it [01:14,  7.47it/s]
No 11 epoch: 554it [01:15,  7.38it/s]
No 12 epoch: 554it [01:15,  7.32it/s]
No 13 epoch: 554it [01:14,  7.47it/s]
No 14 epoch: 554it [01:16,  7.26it/s]
No 15 epoch: 554it [01:17,  7.19it/s]
No 16 epoch: 554it [01:14,  7.39it/s]
/home/ly/miniconda3/envs/torch1.7/lib/python3.8/site-packages/torch/nn/modules/rnn.py:742: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/aten/sr

CPU times: user 14min 9s, sys: 5min 4s, total: 19min 13s
Wall time: 22min 44s


((0.581489841986456, 0.5761250318730099), 0.9582645910588129)